This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [134]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

#print(API_KEY)

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [135]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = "https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?api_key={}".format(API_KEY)

r = requests.get(url)

json_data = r.json()

print(json_data['dataset_data']['data'][5])

['2020-11-24', 116.1, 116.8, 110.5, 113.5, None, 3184.0, 361404.4, None, None, None]


In [136]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

#just quickly making a copy and erasing the 'data' so that this output is more readable here
copy_json = r.json().copy()
copy_json['dataset_data']['data'] = []

copy_json

{'dataset_data': {'limit': None,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2000-06-07',
  'end_date': '2020-12-01',
  'frequency': 'daily',
  'data': [],
  'collapse': None,
  'order': None}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

#### 1. Getting data for only the year 2017

In [137]:
#selecting just the data in the json data
data = json_data['dataset_data']['data']

#using a list comprehension to get data for only the year 2017 by filtering for that particular string
year_2017 = [y for y in data if y[0][0:4]=='2017']

#pring the first 10 elements just to see how it looks
print(year_2017[:10])

[['2017-12-29', 51.76, 51.94, 51.45, 51.76, None, 34640.0, 1792304.0, None, None, None], ['2017-12-28', 51.65, 51.82, 51.43, 51.6, None, 40660.0, 2099024.0, None, None, None], ['2017-12-27', 51.45, 51.89, 50.76, 51.82, None, 57452.0, 2957018.0, None, None, None], ['2017-12-22', 51.05, 51.5, 50.92, 51.32, None, 71165.0, 3641949.0, None, None, None], ['2017-12-21', 51.16, 51.52, 50.9, 51.4, None, 120649.0, 6179433.0, None, None, None], ['2017-12-20', 51.88, 52.04, 51.2, 51.27, None, 50587.0, 2610258.0, None, None, None], ['2017-12-19', 52.73, 52.73, 51.07, 51.66, None, 137313.0, 7102361.0, None, None, None], ['2017-12-18', 52.37, 52.75, 51.61, 52.62, None, 129733.0, 6770499.0, None, None, None], ['2017-12-15', 52.7, 52.7, 51.64, 52.01, None, 204080.0, 10596319.0, None, None, None], ['2017-12-14', 53.11, 53.54, 52.15, 52.67, None, 132981.0, 7016953.0, None, None, None]]


#### 2. Converting data into a dict

In [138]:
# using the dates as the keys and the rest of the data as the values, so the values will be a list of prices, 
#given a particular day
dict_2017 = {day[0]: day[1:] for day in year_2017}

#printing just the first few dictionary elements to see how it looks. 
#Used the code from https://www.geeksforgeeks.org/python-get-first-n-keyvalue-pairs-in-given-dictionary/ to help with this
print(dict(list(dict_2017.items())[0: 3]))

{'2017-12-29': [51.76, 51.94, 51.45, 51.76, None, 34640.0, 1792304.0, None, None, None], '2017-12-28': [51.65, 51.82, 51.43, 51.6, None, 40660.0, 2099024.0, None, None, None], '2017-12-27': [51.45, 51.89, 50.76, 51.82, None, 57452.0, 2957018.0, None, None, None]}


##### 3. Getting highest and lowest opening prices for this period (year 2017)

In [139]:
#getting dict values as a list to iterate over
days_2017 = list(dict_2017.values())

#comprehension list to get the opening prices by iterating over the dictionary values of the year 2017
opening = [day[0] for day in days_2017 if day[0] != None]

print('The highest opening price for the year 2017 is: {}'.format(max(opening)))
print('The lowest opening price for the year 2017 is: {}'.format(min(opening)))

The highest opening price for the year 2017 is: 53.11
The lowest opening price for the year 2017 is: 34.0


#### 4. Finding the largest change in one day based on high and low prices

In [140]:
#using the list of values above again, the high prices are the second value and the low prices are the third
#subtracting these and taking the absolute value will give us the change
daily_changes = [abs(day[1]-day[2]) for day in days_2017]

#Now just take the max
print('The largest change in high and low prices in one day was: {}'.format(max(daily_changes)))

The largest change in high and low prices in one day was: 2.8100000000000023


#### 5. Finding the largest change between any two days based on the closing price

In [141]:
#getting a list of all closing prices
closing_prices = [day[3] for day in days_2017]

#just take the min and max and subtract them to get the largest difference between any two days 
min_close = min(closing_prices)
max_close = max(closing_prices)

max_diff = max_close - min_close

print('The largest change between any two days based on closing price is: {}'.format(max_diff))

The largest change between any two days based on closing price is: 19.03


#### 6. Getting the average of the daily traded volumes 

In [142]:
#getting a list of all trading volumes
trading_volumes = [day[5] for day in days_2017]

#get the average without importing any other libraries
avg_traded_vol = sum(trading_volumes)/len(trading_volumes)

print('The average trading volume for the year 2017 is: {}'.format(avg_traded_vol))

The average trading volume for the year 2017 is: 89124.33725490196


#### 7. Getting the median of the daily traded volumes 

In [143]:
#defining a function to get the median
def median(lst):
    #if list is of even length, then median is the average of the two middle values
    if len(lst) % 2 == 0:
        ind_1 = len(lst)//2
        ind_2 = len(lst)//2 - 1
        
        med_1 = lst[ind_1]
        med_2 = lst[ind_2]
        
        return (med_1+med_2)/2
    #if list is of odd length, then it is much easier as you just take the middle value
    else:
        med = len(lst)//2
        return lst[med]

In [144]:
#Now getting the median is easy, since we have the function, 
#we just need to pass the trading volumes list already created above into it
med_traded_vol = median(trading_volumes)

print('The median trading volume for the year 2017 is: {}'.format(med_traded_vol))

The median trading volume for the year 2017 is: 134965.0
